In [1]:
import dace
from dace.transformation.interstate import GPUTransformSDFG, StateFusion
from dace.transformation.dataflow import MapTiling, InLocalStorage, MapExpansion, MapCollapse, MapReduceFusion
from dace.transformation.optimizer import Optimizer
from dace.transformation import helpers as xfutil

def find_map_by_param(sdfg: dace.SDFG, pname: str) -> dace.nodes.MapEntry:
    """ Finds the first map entry node by the given parameter name. """
    return next((n, state) for n, state in sdfg.all_nodes_recursive()
                if isinstance(n, dace.nodes.MapEntry) and pname in n.params)


def find_map_by_name(sdfg: dace.SDFG, name: str) -> dace.nodes.MapEntry:
    """ Finds the first map entry node by the given parameter name. """
    return next((n, state) for n, state in sdfg.all_nodes_recursive()
                if isinstance(n, dace.nodes.MapEntry) and name == n.label)

M = dace.symbol('M')
N = dace.symbol('N')
K = dace.symbol('K')
alpha = dace.symbol('alpha')
beta = dace.symbol('beta')

@dace.program
def matmul(A: dace.float64[M, K], B: dace.float64[K, N], C: dace.float64[M, N], alpha: dace.float64, beta: dace.float64):
    return alpha * (A @ B) + beta * C

M = 640
N = 640
K = 640
beta = 1
alpha = 1

In [2]:
sdfg = matmul.to_sdfg()
sdfg

Applied 4 StateFusion.


In [3]:
sdfg.apply_transformations(MapReduceFusion)
sdfg

In [4]:
sdfg.apply_transformations(GPUTransformSDFG)
sdfg

Applied 2 StateFusion.
Applied 1 GPUTransformSDFG.


In [ ]:
sdfg.apply_transformations(GPUTransformSDFG)
sdfg

In [7]:
sdfg.expand_library_nodes()
sdfg

Automatically expanded library node "_MatMult_" with implementation "specialize".
Automatically expanded library node "_MatMult_gemm" with implementation "pure".


In [10]:
entry, state = find_map_by_param(sdfg, 'gemm')
divides_evenly = (M % 64 == 0) and (N % 64 == 0) and (K % 8 == 0)
# xfutil.tile(sdfg, entry, divides_evenly, True, i=64, j=64, k=8)
# xfutil.tile(sdfg, entry, divides_evenly, True, i=8, j=4)

StopIteration: 